In [1]:
import DataLoader
import importlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import CoverageHelper
importlib.reload(DataLoader)

%matplotlib inline
test_graph_df = DataLoader.load_csv_test_graphs()
# stat_graph_df = DataLoader.load_csv_stat_graph("")
# stat_trans_graph_df = DataLoader.load_csv_stat_graph("-transitive")
stat_trans_act_graph_df = DataLoader.load_csv_stat_graph("-act-transitive")

Number of apps: 
375


Total entries:
32270


Number of apps: 
375


Total entries:
32270


Total entries:
1937


Total entries:
32270


In [21]:
test_graph_df.head()


,source,target,action,widget_id,widget,strategy,app
0,com.truven.commonandroid.activity.ContentDownl...,NaN,NaN,NaN,NaN,systematic,com.truven.druginfonative.customer
1,com.truven.commonandroid.activity.ContentDownl...,com.truven.druginfonative.customer.MainActivit...,scrollDown,NaN,NaN,random,com.truven.druginfonative.customer
2,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva
3,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva
4,com.startapp.android.publish.OverlayActivity,com.alvines.parivahanseva.MainActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva


In [22]:
# Remove back?
# test_graph_df = test_graph_df[test_graph_df.action != "back"]

In [23]:
# stat_graph_df.head()

In [25]:
lol = stat_trans_act_graph_df[["app", "method", "widget"]]
lol[~lol.widget.notnull()].groupby(["app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count").sort_values(by=["widget_count"], ascending=False)


,,widget_count
app,widget,
com.yinzcam.nfl.packers,NaN,3429
naukriApp.appModules.login,NaN,2223
com.ifs.banking.fiid1603,NaN,717
com.newchic.client,NaN,481
com.paywithoper.oper,NaN,439
com.thomsonreuters.reuterstv,NaN,393
com.youlocal,NaN,369
com.airgoat.goat,NaN,265
com.brisk.jpay,NaN,264


In [10]:
# stat_trans_graph_df.head()

In [11]:
#difference
# pd.concat([stat_trans_graph_df,stat_trans_act_graph_df]).drop_duplicates(keep=False)


In [12]:
unique_widgets_per_app = test_graph_df.groupby(["strategy", "app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")

unique_widgets_per_app.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.funny_photo_editor</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

In [13]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.groupby(level=0).apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="30" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
      <td>0.266430</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.funny_photo_editor</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.love_photo_frames_hd</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>biz.andxor.hearingaid</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">br.com.blackmountain.mylook</th>
      <th>android.support.v7.widget.CardView</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>br.com.geochenapps.tonguetwister</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>br.org.altitudesl</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">brdata.cms.base.foodbazaar</th>
      <th>android.widget.ImageSwitcher</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">bth.studio.quickcharge</th>
      <th>android.widget.FrameLayout</th>
      <td>14</td>
      <td>1.243339</td>
    </tr>
    <tr>
      <th>android.widget.ToggleButton</th>
      <td>6</td>
      <td>0.532860</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">carrioncastillo.invitaciones.happy</th>
      <th>NaN</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>church.itvessel.com.tarm</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.appzone.www.eye_makeup</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.instavr.iostest</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.mobius.vrcinema</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>coloring.book.color</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.BillDirkes.QuickEM</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.CarsLiveWallpaperHQ</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.CrochetPatternIdeas.BehMedia</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">com.Feng.LingCrazyLockScreen</th>
      <th>android.widget.Button

In [14]:
number_of_unique_screens_per_app = test_graph_df[['strategy', 'app', 'source', 'target', 'widget']].melt(
    id_vars=['strategy', 'app', 'widget'])
number_of_unique_screens_per_app

,strategy,app,widget,variable,value
0,systematic,com.learndrawing.howtodrawfnaf,NaN,source,com.learndrawing.howtodrawfnaf.MainActivity
1,random,com.learndrawing.howtodrawfnaf,NaN,source,com.learndrawing.howtodrawfnaf.MainActivity
2,systematic,com.sole.insider.free,NaN,source,com.sole.insider.free.SoleInsider
3,random,com.sole.insider.free,NaN,source,com.sole.insider.free.SoleInsider
4,systematic,com.CarsLiveWallpaperHQ,NaN,source,com.lpp.MainActivity
5,random,com.CarsLiveWallpaperHQ,NaN,source,com.lpp.MainActivity
6,systematic,com.vadevteam.thandongdatviet,android.widget.Button,source,com.vadevteam.thandongdatviet.SplashScreen
7,systematic,com.vadevteam.thandongdatviet,NaN,source,com.vadevteam.thandongdatviet.SplashScreen
8,systematic,com.vadevteam.thandongdatviet,android.widget.Button,source,com.vadevteam.thandongdatviet.SplashScreen
9,systematic,com.vadevteam.thandongdatviet,NaN,source,com.vadevteam.thandongdatviet.SplashScreen


Edges from

In [2]:
systematic_test_graph = test_graph_df[test_graph_df.strategy == "systematic"]
systematic_test_graph.head()

,source,target,action,widget_id,widget,strategy,app
0,com.truven.commonandroid.activity.ContentDownl...,NaN,NaN,NaN,NaN,systematic,com.truven.druginfonative.customer
2,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva
3,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva
4,com.startapp.android.publish.OverlayActivity,com.alvines.parivahanseva.MainActivity,back,NaN,NaN,systematic,com.alvines.parivahanseva
8,com.newspress.hurricanehub.MainActivity,NaN,NaN,NaN,NaN,systematic,com.newspress.hurricanehub


In [3]:
random_test_graph = test_graph_df[(test_graph_df.strategy == "random")]
random_test_graph.head()


,source,target,action,widget_id,widget,strategy,app
1,com.truven.commonandroid.activity.ContentDownl...,com.truven.druginfonative.customer.MainActivit...,scrollDown,NaN,NaN,random,com.truven.druginfonative.customer
5,com.startapp.android.publish.OverlayActivity,com.alvines.parivahanseva.MainActivity,back,NaN,NaN,random,com.alvines.parivahanseva
6,com.alvines.parivahanseva.MainActivity,com.alvines.parivahanseva.WebViewActivity_yahoo,click,2.131100e+09,android.widget.Button,random,com.alvines.parivahanseva
7,com.alvines.parivahanseva.WebViewActivity_yahoo,com.startapp.android.publish.OverlayActivity,back,NaN,NaN,random,com.alvines.parivahanseva
9,com.newspress.hurricanehub.MainActivity,NaN,NaN,NaN,NaN,random,com.newspress.hurricanehub


Systematic:

In [12]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, systematic_test_graph)
direct_edges_not_found.head()

,package,source,target,method,widget,app
0,com.getabby.ridecats,com.getabby.ridecats.view.ActivityMain,com.getabby.ridecats.view.MenuActivity,onClickNavFavorites,android.widget.ImageView,com.getabby.ridecats
1,com.getabby.ridecats,com.getabby.ridecats.view.ActivityMain,com.getabby.ridecats.view.MenuActivity,onClickNavFeedback,android.widget.ImageView,com.getabby.ridecats
2,com.getabby.ridecats,com.getabby.ridecats.view.ActivityMain,com.getabby.ridecats.view.LiveTrackerFragment,onNewIntent,NaN,com.getabby.ridecats
3,com.getabby.ridecats,com.getabby.ridecats.view.ActivityMain,com.getabby.ridecats.common.activities.BaseAct...,onCreateView,android.view.ViewGroup,com.getabby.ridecats
4,com.getabby.ridecats,com.getabby.ridecats.view.ActivityMain,com.getabby.ridecats.common.activities.BaseAct...,onCreateView,android.view.LayoutInflater,com.getabby.ridecats


widgets from missed edges from systematic test graphs

In [18]:
# unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
# unique_widgets_per_app.head()

unique_widgets_per_app = direct_edges_not_found.widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app

,widget_count
NaN,1403
android.view.MenuItem,265
android.view.View,251
android.widget.ListView,200
android.widget.ImageView,181
android.widget.Button,171
android.app.AlertDialog$Builder,105
android.widget.TextView,92
android.widget.RelativeLayout,75
android.widget.PopupMenu,57


In [19]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False)

,widget_count,percentage
NaN,1403,34.753530
android.view.MenuItem,265,6.564280
android.view.View,251,6.217488
android.widget.ListView,200,4.954174
android.widget.ImageView,181,4.483527
android.widget.Button,171,4.235819
android.app.AlertDialog$Builder,105,2.600941
android.widget.TextView,92,2.278920
android.widget.RelativeLayout,75,1.857815
android.widget.PopupMenu,57,1.411940


Random:

In [4]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
direct_edges_not_found.head()

,package,source,target,method,widget,app
0,com.alvines.parivahanseva,com.alvines.parivahanseva.MainActivity,com.alvines.parivahanseva.WebViewActivity,onClick,android.widget.Button,com.alvines.parivahanseva
1,com.alvines.parivahanseva,com.alvines.parivahanseva.MainActivity,com.alvines.parivahanseva.WebViewActivity_gmail,onClick,android.widget.Button,com.alvines.parivahanseva
2,com.lauapps.decorhome,com.lauapps.decorhome.DecorStylesActivity,com.lauapps.decorhome.DecorStylesProfile,onListItemClick,android.view.View,com.lauapps.decorhome
3,com.lauapps.decorhome,com.lauapps.decorhome.MainActivity2,com.lauapps.decorhome.DecorStylesProfile2,onListItemClick,android.view.View,com.lauapps.decorhome
4,com.lauapps.decorhome,com.lauapps.decorhome.MainActivity2,com.lauapps.decorhome.j,onItemClick,android.widget.ListView,com.lauapps.decorhome


In [15]:
unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
unique_widgets_per_app[["widget", "widget_count"]].sort_values(by=["widget_count"], ascending=False)

,widget,widget_count
459,NaN,286
899,android.view.MenuItem,175
300,android.widget.ImageView,129
263,NaN,122
930,NaN,109
301,android.widget.Button,89
27,NaN,74
302,android.view.View,65
303,android.widget.RelativeLayout,64
28,android.widget.ListView,55


In [22]:
unique_widgets_per_app.count()

widget_count    1226
percentage      1226
dtype: int64